In [1]:
import sys
sys.path.append('/Applications/anaconda/lib/python2.7/site-packages')

In [2]:
import numpy as np
import pandas as pd
import nltk

In [3]:
train = pd.read_csv('train.csv',encoding="ISO-8859-1")
test = pd.read_csv('test.csv',encoding="ISO-8859-1")
description = pd.read_csv('product_descriptions.csv',encoding="ISO-8859-1")

In [61]:
train.head(3)

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0


In [12]:
test.head(3)

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able


In [13]:
description.head(3)

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...


In [198]:
pattern = r'''\b\$?\d+(\.\d+)?%?\b       # special characters with meanings
            '''

In [4]:
from nltk.corpus import stopwords # delete nonsense words like 'a' 'the'
from nltk.tokenize import RegexpTokenizer # split the sentence by break
from nltk.stem.snowball import SnowballStemmer # stemmer words
from nltk.stem.wordnet import WordNetLemmatizer # lemmatize
from nltk.corpus import wordnet
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer('english')
st = WordNetLemmatizer()

In [5]:
def ngrams(input_list, n):
  return zip(*[input_list[i:] for i in range(n)])

In [6]:
def cleanTypo(word):
    if len(word) <= 2:
        return word.split(' ')
    else:
        isTypo = False
        index = 0
        char_list = list(word)
        three_chars = ngrams(char_list,3)
        for i in range(len(three_chars)):
            c1 = 'a' <= three_chars[i][0] <= 'z'
            c2 = 'A' <= three_chars[i][1] <= 'Z'
            c3 = 'a' <= three_chars[i][0] <= 'z'
            isTypo = isTypo or c1&c2&c3
            if isTypo:
                index = i+1
                cor_word = word[:index] + " " + word[index:]
                break

            else:
                cor_word = word


        return cor_word.split(" ")

In [7]:
def syno(word):
    syn = wordnet.synsets(word)
    if syn:
        return str(syn[0].lemma_names()[0])
    else:
        return word

In [33]:
description.product_description[5]

u"Achieving delicious results is almost effortless with this Whirlpool over-the-range microwave hood with convection cooking. With 1.9 cu. ft. capacity, you'll have room to prepare multiple dishes at once. And, you won't have to micro-manage the cooking process thanks to the sensor cooking options. Sensor cooking tracks cooking progress and automatically adjusts cooking time. Convection roasting and baking allows you to bake cakes, bread, cookies and roast meats with ease. Clean-up is simple too with the industry leading CleanRelease non-stick interior surface. It easily releases cooked-on food without the use of harsh chemicals. A damp cloth or sponge is all that is needed to remove cooked-on spills and splashes.California residents: see&nbsp;Proposition 65 informationSpacious 1.9 cu. ft. capacity accommodates dinner plates and casserole dishes with ease1100 watts of cooking power and 10 cooking levels make cooking and reheating a snap400 CFM venting system whisks smoke, steam and odo

In [ ]:
def generateList(item):
    textList = item.split(' ')

In [8]:
def generateList(item):
    textList = [word for word in tokenizer.tokenize(item) if word not in stopwords.words('english')]
    text = []
    for i, item in enumerate(textList):
        text = text + cleanTypo(item)
    clean_text = [stemmer.stem(st.lemmatize(syno(word),'v')) for word in text]
    return clean_text

In [5]:
def generateDic(item):
    text = " ".join([st.lemmatize(stemmer.stem(word),'v') for word in tokenizer.tokenize(item.lower()) if word not in stopwords.words('english')])
    dic = dict()
    for w in text.split():
        if w in dic.keys():
            dic[w] = dic[w]+1
        else:
            dic[w] = 1
    return dic

In [9]:
train['productList']=train['product_title'].map(lambda x: generateList(x))

In [10]:
train['searchList']=train['search_term'].map(lambda x: generateList(x))

In [11]:
train.head(3)

,id,product_uid,product_title,search_term,relevance,productList,searchList
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,"[simpson, strong, neckti, twelv, gaug, angl]","[angl, bracket]"
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,"[simpson, strong, neckti, twelv, gaug, angl]","[liter, bracket]"
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0,"[behr, premium, textur, deck, over, one, gallo...",[deck]


In [12]:
test['productList']=test['product_title'].map(lambda x: generateList(x))

In [13]:
test['searchList']=test['search_term'].map(lambda x: generateList(x))

In [14]:
test.head(3)

,id,product_uid,product_title,search_term,productList,searchList
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"[simpson, strong, neckti, twelv, gaug, angl]","[nineti, degre, bracket]"
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"[simpson, strong, neckti, twelv, gaug, angl]","[metallic_el, liter, bracket]"
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"[simpson, strong, neckti, twelv, gaug, angl]","[simpson, sku, abl]"


In [15]:
description['desList']=description['product_description'].map(lambda x: generateList(x))

In [16]:
description.head(3)

,product_uid,product_description,desList
0,100001,"Not only do angles make joints stronger, they ...","[not, angl, brand, joint, strong, besid, suppl..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...,"[behr, premium, textur, deckov, advanc, solid,..."
2,100003,Classic architecture meets contemporary design...,"[classic, architectur, meet, contemporari, des..."


In [17]:
train_clean = pd.merge(train,description,how='left',on='product_uid')
column_order=['id','product_uid','product_title','search_term','product_description','relevance', 'productList','searchList','desList']
train_clean = train_clean[column_order]

In [18]:
train_clean.head(3)

,id,product_uid,product_title,search_term,product_description,relevance,productList,searchList,desList
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,"Not only do angles make joints stronger, they ...",3.0,"[simpson, strong, neckti, twelv, gaug, angl]","[angl, bracket]","[not, angl, brand, joint, strong, besid, suppl..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,"Not only do angles make joints stronger, they ...",2.5,"[simpson, strong, neckti, twelv, gaug, angl]","[liter, bracket]","[not, angl, brand, joint, strong, besid, suppl..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,BEHR Premium Textured DECKOVER is an innovativ...,3.0,"[behr, premium, textur, deck, over, one, gallo...",[deck],"[behr, premium, textur, deckov, advanc, solid,..."


In [19]:
test_clean = pd.merge(test,description,how='left',on='product_uid')
column_order=['id','product_uid','product_title','search_term','product_description','productList','searchList','desList']
test_clean = test_clean[column_order]

In [20]:
test_clean.head(3)

,id,product_uid,product_title,search_term,product_description,productList,searchList,desList
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket,"Not only do angles make joints stronger, they ...","[simpson, strong, neckti, twelv, gaug, angl]","[nineti, degre, bracket]","[not, angl, brand, joint, strong, besid, suppl..."
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets,"Not only do angles make joints stronger, they ...","[simpson, strong, neckti, twelv, gaug, angl]","[metallic_el, liter, bracket]","[not, angl, brand, joint, strong, besid, suppl..."
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able,"Not only do angles make joints stronger, they ...","[simpson, strong, neckti, twelv, gaug, angl]","[simpson, sku, abl]","[not, angl, brand, joint, strong, besid, suppl..."


In [21]:
train_clean.to_csv('train_clean.csv',index=False,encoding="ISO-8859-1")
test_clean.to_csv('test_clean.csv',index=False,encoding="ISO-8859-1")

## Generate Counting Features

In [22]:
train_clean.head(3)

,id,product_uid,product_title,search_term,product_description,relevance,productList,searchList,desList
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,"Not only do angles make joints stronger, they ...",3.0,"[simpson, strong, neckti, twelv, gaug, angl]","[angl, bracket]","[not, angl, brand, joint, strong, besid, suppl..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,"Not only do angles make joints stronger, they ...",2.5,"[simpson, strong, neckti, twelv, gaug, angl]","[liter, bracket]","[not, angl, brand, joint, strong, besid, suppl..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,BEHR Premium Textured DECKOVER is an innovativ...,3.0,"[behr, premium, textur, deck, over, one, gallo...",[deck],"[behr, premium, textur, deckov, advanc, solid,..."


In [23]:
def countNum(dictionary):
    total = 0
    for key in dictionary:
        if key.isdigit():
            total += dictionary[key]
    return total

In [24]:
train_count = pd.DataFrame()

In [25]:
train_count['count_product'] = train_clean['productList'].map(lambda x: len(x))

In [26]:
train_count['count_search'] = train_clean['searchList'].map(lambda x: len(x))

In [27]:
train_count['count_des'] = train_clean['desList'].map(lambda x: len(x))

In [28]:
train_count['countNum_product'] = train_clean['productList'].map(lambda x: countNum(x))

TypeError: list indices must be integers, not unicode

In [ ]:
train_count['countNum_search'] = train_clean['searchList'].map(lambda x: countNum(x))

In [ ]:
train_count['countNum_des'] = train_clean['desList'].map(lambda x: countNum(x))